## AWS EC2 with Boto3 Python Tutorial 

https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html

Amazon Elastic Compute Cloud (Amazon EC2) is a web service that provides resizeable computing capacity in servers in Amazon's data centers—that you use to build and host your software systems.

- Create Key-Pair for Secure Connection
- Create an Amazon EC2 instance
    - test with ip address in browser
    - test with ssh connection
- Create Security Group and add rules to it
- Attaching and Detaching Security Groups to Amazon EC2 instances
    - test with ip address in browser
    - test with ssh connection
- Start, Stop and Delete Amazon EC2 instances

## Create Key-Pair for Secure Connection

In [1]:
import boto3

In [ ]:
ec2 = boto3.client('ec2')

ec2.describe_instances()

resp = ec2.create_key_pair(KeyName='my-key-pair')
# print(resp['KeyMaterial'])

In [7]:
file = open("Creds/my-key-pair.pem", 'w')
file.write(resp['KeyMaterial'])
file.close()

easy way of creating the keypair and loading it here rather than manually creating and download .pem file if you see AD1.pem is the one manually created and downloaded which we are going to use it here

<!--easy way of creating keypair and loading here rather than manually creating and download  -->

## Create an Amazon EC2 instance

In [ ]:
ec2.describe_instances() #to check what does it hold before passing the necessary parameters.

In [9]:
ec2.describe_instances()

response = ec2.run_instances(
    ImageId = 'ami-05ffe3c48a9991133',  #  AMI ID for Ubuntu 20.04 in us-east-1 changes basis the OS yo want to use i have choosen aws linux 
    MinCount=1,
    MaxCount=1,
    InstanceType='t2.micro',
    KeyName='my-key-pair',
    BlockDeviceMappings=[
        {
            "DeviceName": "/dev/xvda",
            'Ebs':{
                'DeleteOnTermination': True, # if you want to delete the volume when instance is terminated 
                'VolumeSize': 20    # size of the volume in GB free tier is 30 GB 
            }
        }
    ]

)

In [ ]:
response

## Create Security Group and add rules to it
- Security groups control inbound and outbound traffic of the EC2 instance network interface.
- every EC2 instance must have at least one Security Group associated with it. If no Security Group has been specified during the EC2 instance launch, the default Security Group of the default VPC is associated with the instance.

In [12]:
response = ec2.describe_security_groups()

response = ec2.create_security_group(
    GroupName = 'ADGroupname',
    Description = "Security group for testing purpose"
)

In [13]:
security_group_id = response['GroupId']
security_group_id

'sg-05583770644e80787'

In [15]:
# ip, port, traffic type
response = ec2.authorize_security_group_ingress(
    GroupId = security_group_id,
    IpPermissions=[
        {
            'IpProtocol': 'tcp',
            'FromPort': 22,
            'ToPort': 22,
            'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
        },
        {
            'IpProtocol': 'tcp',
            'FromPort': 80,
            'ToPort': 80,
            'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
        }
    ]
)

In [ ]:
response = ec2.describe_instances()

instance_id = response['Reservations'][0]['Instances'][0]['InstanceId']
instance_id, security_group_id

('i-0929bd4287a6af5b2', 'sg-05583770644e80787')

In [17]:
old_gid = response['Reservations'][0]['Instances'][0]['SecurityGroups'][0]['GroupId']

In [ ]:
ec2.modify_instance_attribute(InstanceId=instance_id, Groups=[security_group_id])

In [ ]:
ec2.modify_instance_attribute(InstanceId=instance_id, Groups=[old_gid, security_group_id])

## Start, Stop and Delete Amazon EC2 instances

In [ ]:
# status -> running, stopped, terminated, pending etc.

import time

def wait_for_status(instance_id, target_status):
    while True:
        response = ec2.describe_instances(InstanceIds=instance_id)

        status = response['Reservations'][0]['Instances'][0]['State']['Name']

        if status == target_status:
            print("Instance is in {} state".format(target_status))
            break
        
        time.sleep(10)


def start_instances(instance_id):
    print("EC2 Instance Start")
    ec2.start_instances(InstanceIds=instance_id)

    wait_for_status(instance_id, 'running')

start_instances([instance_id])


EC2 Instance Start
Instance is in running state


In [ ]:
def stop_instances(instance_id):
    print("EC2 Instance Stop")
    ec2.stop_instances(InstanceIds=instance_id)

    wait_for_status(instance_id, 'stopped')

stop_instances([instance_id])

EC2 Instance Stop
Instance is in stopped state


In [ ]:
def terminate_instances(instance_id):
    print("EC2 Instance Termination")
    ec2.terminate_instances(InstanceIds=instance_id)

    wait_for_status(instance_id, 'terminated')

terminate_instances([instance_id])

EC2 Instance Termination
Instance is in terminated state
